In [1]:
import nltk

nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [2]:
import tensorflow as tf

keras = tf.keras

In [3]:
%matplotlib inline
import re
import seaborn as sbn
import nltk
import tqdm as tqdm
import sqlite3
import pandas as pd
import numpy as np

import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from nltk.stem import SnowballStemmer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

from sklearn.svm import LinearSVC
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers import LSTM, Dense, Embedding
from keras.models import Model


from gensim import summarization
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('Reviews1.csv')

In [5]:
review_data = df[['UserId', 'ProductId', 'Score','Summary','Text']]
review_data.head()

,UserId,ProductId,Score,Summary,Text
0,A3SGXH7AUHU8GW,B001E4KFG0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,A1D87F6ZCVE5NK,B00813GRG4,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,ABXLMWJIXXAIN,B000LQOCH0,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,A395BORC6FGVXV,B000UA0QIQ,2,Cough Medicine,If you are looking for the secret ingredient i...
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
review_data['UserId'].value_counts()

A3NHUQ33CFH3VM    5
A3PJZ8TU8FDQ1K    5
A2NLZ3M0OJV9NX    4
A3RMGIKUWGPZOK    4
A31N6KB160O508    3
                 ..
A3MA4VWLFOCJ41    1
A2NT2JB4DBPPVP    1
A2FOXRMJA0T9M7    1
A3TXK9O5IP3CWQ    1
A1BQK9Q5K8MHWM    1
Name: UserId, Length: 2890, dtype: int64

In [7]:

# user1_data = review_data.loc[review_data['UserId'] == 'A3NHUQ33CFH3VM']
labels = review_data['Score'].map(lambda x : 1 if int(x) > 3 else 0)
print(np.array(labels))

[1 0 1 ... 1 0 1]


In [8]:
review_data['Score'] = review_data['Score'].map(lambda x : 1 if int(x) > 3 else 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
review_data.head()

,UserId,ProductId,Score,Summary,Text
0,A3SGXH7AUHU8GW,B001E4KFG0,1,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,A1D87F6ZCVE5NK,B00813GRG4,0,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,ABXLMWJIXXAIN,B000LQOCH0,1,"""Delight"" says it all",This is a confection that has been around a fe...
3,A395BORC6FGVXV,B000UA0QIQ,0,Cough Medicine,If you are looking for the secret ingredient i...
4,A1UQRSCLF8GW1T,B006K2ZZ7K,1,Great taffy,Great taffy at a great price. There was a wid...


In [10]:
def data_clean(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [11]:
clean_reviewData = []
for rev in review_data['Text']:
    clean_reviewData.append( " ".join(data_clean(rev)))

clean_summaryData = []
for rev in review_data['Summary']:
    clean_summaryData.append( " ".join(data_clean(rev)))

In [12]:
Data = review_data[['Text','Score']]
print(Data.head())
Data_x= Data['Text']
Data_y= Data['Score']
#calculate padding length
MAX_LEN = 0
for i in range(1,2999):
  if(MAX_LEN < len(Data_x[i])):
    MAX_LEN = len(Data_x[i])
print(MAX_LEN)

                                                Text  Score
0  I have bought several of the Vitality canned d...      1
1  Product arrived labeled as Jumbo Salted Peanut...      0
2  This is a confection that has been around a fe...      1
3  If you are looking for the secret ingredient i...      0
4  Great taffy at a great price.  There was a wid...      1
5276


In [13]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(Data_x)
sequences = tokenizer.texts_to_sequences(Data_x)
data = pad_sequences(sequences, maxlen=MAX_LEN)
print(data.shape)

(2999, 5276)


In [14]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-11-22 17:51:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-11-22 17:51:20--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-11-22 17:51:20--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [15]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [16]:
#using glove model for word embeddings
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [17]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

print(embedding_matrix[1])

[ 4.65600006e-02  2.13180006e-01 -7.43639981e-03 -4.58539993e-01
 -3.56389992e-02  2.36430004e-01 -2.88360000e-01  2.15210006e-01
 -1.34859994e-01 -1.64129996e+00 -2.60910004e-01  3.24340016e-02
  5.66210002e-02 -4.32960019e-02 -2.16719992e-02  2.24759996e-01
 -7.51290023e-02 -6.70180023e-02 -1.42470002e-01  3.88250016e-02
 -1.89510003e-01  2.99769998e-01  3.93049985e-01  1.78870007e-01
 -1.73429996e-01 -2.11779997e-01  2.36169994e-01 -6.36809990e-02
 -4.23180014e-01 -1.16609998e-01  9.37540010e-02  1.72959998e-01
 -3.30729991e-01  4.91120011e-01 -6.89949989e-01 -9.24620032e-02
  2.47419998e-01 -1.79910004e-01  9.79079977e-02  8.31179991e-02
  1.52989998e-01 -2.72760004e-01 -3.89339998e-02  5.44529974e-01
  5.37370026e-01  2.91049987e-01 -7.35139987e-03  4.78800014e-02
 -4.07599986e-01 -2.67590005e-02  1.79189995e-01  1.09770000e-02
 -1.09630004e-01 -2.63949990e-01  7.39900023e-02  2.62360007e-01
 -1.50800005e-01  3.46230000e-01  2.57580012e-01  1.19709998e-01
 -3.71350013e-02 -7.15930

In [18]:
#create the model with CNN 5 filters are used and with a hidden layer and non trainable embedding layer
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
  embedding_layer = Embedding(num_words, embedding_dim, weights=[embeddings],
                              input_length=max_sequence_length, trainable=False)
  sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
  embedded_sequences = embedding_layer(sequence_input)

  convs = []
  filter_sizes = [2,3,4,5,6]

  for filter_size in filter_sizes:
    l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
    l_pool = GlobalMaxPooling1D()(l_conv)
    convs.append(l_pool)
  
  l_merge = concatenate(convs, axis=1)
  x = Dropout(0.1)(l_merge)
  x = Dense(128, activation='relu')(x)
  x = Dropout(0.2)(x)
  preds = Dense(1, activation='sigmoid')(x)

  model = Model(sequence_input, preds)
  model.compile(loss='binary_crossentropy', optimizer='adam',
                metrics=['mse'])
  model.summary()
  return model

In [19]:
label_names = ['1','0']
model = ConvNet(embedding_matrix, 50, vocabulary_size, 300, len(list(label_names)))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      6000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 200)      120200      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      180200      embedding[0][0]                  
_______________________________________________________________________________________

In [20]:
history = model.fit(data, np.array(labels), validation_split=0.4, epochs = 3)

Epoch 1/3
57/57 [==============================] - 21s 366ms/step - loss: 0.5435 - mse: 0.1687 - val_loss: 0.5433 - val_mse: 0.1843
Epoch 2/3
57/57 [==============================] - 20s 360ms/step - loss: 0.3729 - mse: 0.1167 - val_loss: 0.4796 - val_mse: 0.1591
Epoch 3/3
57/57 [==============================] - 21s 363ms/step - loss: 0.1807 - mse: 0.0491 - val_loss: 0.4792 - val_mse: 0.1524


In [42]:
sentence = "Worst"
clean_reviewData = []
clean_reviewData.append( " ".join(data_clean(sentence)))

In [43]:
print(clean_reviewData)

['w o r s t']


In [44]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(clean_reviewData)
sequences = tokenizer.texts_to_sequences(clean_reviewData)
final_string = pad_sequences(sequences, maxlen=MAX_LEN)
print(final_string.shape)

(1, 5276)


In [45]:
print(final_string)
what= ''
if (round(float(model.predict(final_string)))==1):
    what= 'Positive'
    acc= round(float(model.predict(final_string))*100,2)
else:
    what='Negative'
    acc= 100- round(float(model.predict(final_string))*100,2)
print(what,'review with',acc,'% Accuracy')

[[0 0 0 ... 3 4 5]]
Positive review with 77.24 % Accuracy
